# 設定
'''
miniforge create -n env_chroma chromadb
Python 3.11.7
## conda environments:
                            C:\Users\onowa\miniconda3
                            C:\Users\onowa\miniconda3\envs\vir_env
    base                     C:\Users\onowa\miniforge3
    env_chroma            *  C:\Users\onowa\miniforge3\envs\env_chroma
    py310                    C:\Users\onowa\miniforge3\envs\py310
    vir_env                  C:\Users\onowa\miniforge3\envs\vir_env

# packages in environment
    langchain                 0.0.353            pyhd8ed1ab_0    conda-forge
    langchain-community       0.0.7              pyhd8ed1ab_0    conda-forge
    langchain-core            0.1.4              pyhd8ed1ab_0    conda-forge
    chromadb                  0.4.21          py311h1ea47a8_0    conda-forge

In [1]:
!python -V
# !conda env list
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
# !conda list langchain
# !conda list chromadb

Python 3.11.7


In [3]:
# これを使う
# markdown_splitterでマークダウンファイルを読み込んで、RecursiveCharacterTextSplitter でテキストスプリットする。
# markdown_document = "# Intro \n\n    ## History \n\n Markdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9] \n\n Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files. \n\n ## Rise and divergence \n\n As Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for \n\n additional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks. \n\n #### Standardization \n\n From 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort. \n\n ## Implementations \n\n Implementations of Markdown are available for over a dozen programming languages."
from langchain.text_splitter import MarkdownHeaderTextSplitter
from typing import Any

path = './text/frielen_ja.txt' # "./README.md"
with open(path) as f:
    md = f.read()

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md)  # (markdown_document)

# Char-level splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# japanese_spliter
class JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n","。","　"]
        super().__init__(separators=separators, **kwargs)

text_splitter = JapaneseCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=0,
)

# Split
documents = text_splitter.split_documents(md_header_splits)
print(len(documents))
documents[0]

105


Document(page_content='魔王を倒して王都に凱旋した勇者ヒンメル、僧侶ハイター、戦士アイゼン、魔法使いフリーレンら勇者パーティー4人は、10年間もの旅路を終えて感慨にふけっていたが、1000年は軽く生きる長命種のエルフであるフリーレンにとって、その旅はきわめて短いものであった', metadata={'Header 1': 'あらすじ１'})

In [21]:
# 使わない
# JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter)

loader = DirectoryLoader(
    "./text/", 
    glob="**/*.txt", 
    loader_cls=TextLoader
    , loader_kwargs={'autodetect_encoding': True}
)
data = loader.load()
# print(data)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Any
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# japanese_spliter
class JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n","。","　"]
        super().__init__(separators=separators, **kwargs)

text_splitter = JapaneseCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=0,
)
        
'''
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 11,   # チャンクの文字数
    chunk_overlap  = 0,  # チャンクオーバーラップの文字数
)

text_splitter = CharacterTextSplitter(
    separator= ["\n\n", "\n"],
    chunk_size=900,
    chunk_overlap=0,
    length_function=len
)
'''

documents = text_splitter.create_documents([
    doc.page_content for doc in data])
print(data[0])

page_content='# あらすじ１\n魔王を倒して王都に凱旋した勇者ヒンメル、僧侶ハイター、戦士アイゼン、魔法使いフリーレンら勇者パーティー4人は、10年間もの旅路を終えて感慨にふけっていたが、1000年は軽く生きる長命種のエルフであるフリーレンにとって、その旅はきわめて短いものであった。\nそして、50年に一度降るという「半世紀（エーラ）流星」を見た4人は、次回もそれを見る約束を交わしてパーティーを解散する。\n\n# あらすじ２\n50年後、すっかり年老いたヒンメルと再会したフリーレンは、ハイターやアイゼンとも連れ立って再び流星群を観賞する。\nまもなくヒンメルは亡くなるが、彼の葬儀でフリーレンは自分がヒンメルについて何も知らず、知ろうともしなかったことに気付いて涙する。\nその悲しみに困惑したフリーレンは、人間を知るためと魔法収集のために旅に出る。\n\n# あらすじ３\nそれから20年後、フリーレンは老い先短いハイターを訪ねる。\nハイターは魔導書の解読と戦災孤児のフェルンを弟子にすることをフリーレンに依頼。\nその4年後、フリーレンは魔導書の解読を終え、フェルンは1人前の魔法使いとなる。\nハイターの最期を看取った2人は諸国を巡る旅に出発する。\n\n# あらすじ４\nさらにその後、アイゼンの許を訪れたフリーレン達は、彼の協力によりフリーレンの師匠で伝説の大魔法使いフランメの手記を入手、死者の魂と対話できる場所・オレオールの存在を知る。\nその在処は大陸の北端エンデ。\nかつての魔王城の所在地だった。\n当ての無かった旅は、ヒンメルとの再会を目指す北への旅となる。\nそしてアイゼンの弟子シュタルクや僧侶ザインを仲間に加え、フリーレン達の旅は続く。\n\n\n# 第17話 葬送のフリーレン「じゃあ元気で」\nザインは親友の戦士ゴリラの手掛かりを得るも、彼が向かった先はフリーレンたちの目的地とは別の道だった。\nそんな中、4人が滞在する集落に寒波が到来し、ひと月の足止めを余儀なくされる。\nそれぞれの過ごし方で日々を楽しんでいたが、ある日フェルンとシュタルクがけんかをし、フリーレンはザインを頼る。\n\n\n# 第16話 葬送のフリーレン「長寿友達」\n4人は、かつてフリーレンがヒンメルたちと冒険をしていた時に出会ったドワーフの戦士・フォルじいを訪ねる

In [ ]:
# つかわない
# 単純にMarkdownHeaderTextSplitterを適用する
from langchain.text_splitter import MarkdownHeaderTextSplitter
from typing import Any

path = './text/frielen_ja.txt' # "./README.md"
with open(path) as f:
    md = f.read()

headers_to_split_on = [("#", "Header 1"),
                       ("##", "Header 2"),
                       ("###", "Header 3"),
                       ("####", "Header 4"),
                       ]

# japanese_spliter
class M_JapaneseCharacterTextSplitter(MarkdownHeaderTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n","。","　"]
        super().__init__(separators=separators, **kwargs)

# markdown_splitter = M_JapaneseCharacterTextSplitter(
markdown_splitter = MarkdownHeaderTextSplitter(
         # chunk_size=40,
         # chunk_size=200, chunk_overlap=0,
         headers_to_split_on=headers_to_split_on,
     return_each_line=True
)
documents = markdown_splitter.split_text(md)
for text in documents:
    print(text)

In [69]:
# embeddingの実行
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import os

def database(**args_dict):
    print(args_dict['db'], args_dict['embedding_function'])
    # create the open-source embedding function
    embedding_function = SentenceTransformerEmbeddings(
        model_name = args_dict['embedding_function'],
        encode_kwargs={"normalize_embeddings":False })
    path = os.path.join('./database/', args_dict['db'])
    print(path)
    db = Chroma.from_documents(
        documents=documents,
        embedding = embedding_function,
        persist_directory = path
    )
    if db:
        db.persist()
        # db = None
    else:
        print("Chroma DB has not been initialized.")

list_db = [
    {'db':'db1', 'embedding_function':'sentence-transformers/all-MiniLM-L6-v2'},
    {'db':'db2', 'embedding_function':'oshizo/sbert-jsnli-luke-japanese-base-lite'},
    {'db':'db3', 'embedding_function':'intfloat/multilingual-e5-small'},
    {'db':'db4', 'embedding_function':'sonoisa/sentence-bert-base-ja-mean-tokens-v2'}
]

for dict_db in list_db:
    database(**dict_db)

# embedding_function = SentenceTransformerEmbeddings(model_name=sentence-transformers/all-MiniLM-L6-v2')
# embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite"
# embedding = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-small'
# embedding = HuggingFaceEmbeddings(model_name = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'
# database(**dicttest)

db1 sentence-transformers/all-MiniLM-L6-v2
./database/db1
db2 oshizo/sbert-jsnli-luke-japanese-base-lite
./database/db2
db3 intfloat/multilingual-e5-small
./database/db3
db4 sonoisa/sentence-bert-base-ja-mean-tokens-v2
./database/db4


In [74]:
# 一応、これで完成？
# dbs = ['db1','db2','db3','db4']
# データベースをディスクからロードして、クエリで検索する

list_db = [
    {'db':'db1', 'embedding_function':'sentence-transformers/all-MiniLM-L6-v2'},
    {'db':'db2', 'embedding_function':'oshizo/sbert-jsnli-luke-japanese-base-lite'},
    {'db':'db3', 'embedding_function':'intfloat/multilingual-e5-small'},
    {'db':'db4', 'embedding_function':'sonoisa/sentence-bert-base-ja-mean-tokens-v2'}
]

query ='ヒンメルの旅'
top_k=10

def load_from_disk(**args_dict):
    print(args_dict['db'], args_dict['embedding_function'])
    path = os.path.join('./database/', args_dict['db'])
    # print(path)
    embedding_function = SentenceTransformerEmbeddings(model_name=args_dict['embedding_function'])
    db = Chroma(persist_directory = path, embedding_function=embedding_function)
    # docs = db.similarity_search("フリーレン", top_k=2)
    # docs = db.similarity_search_with_score("フリーレン達の旅", K=top_k)
    docs = db.similarity_search(query, k=top_k)
    # print(docs[0].page_content)
    print(len(docs))
    # similarity_search_with_score の場合：
    # print(f"content: {docs[0][0].page_content}", f"score: {docs[0][1]}")  
    # print(f"content: {docs[3][0].page_content}", f"score: {docs[1][1]}")  
    for mykey in docs:
        print(mykey)
    print('------\n')

for each_db in list_db:
    load_from_disk(**each_db)

db1 sentence-transformers/all-MiniLM-L6-v2
10
page_content='当ての無かった旅は、ヒンメルとの再会を目指す北への旅となる。' metadata={'Header 1': 'あらすじ４'}
page_content='当ての無かった旅は、ヒンメルとの再会を目指す北への旅となる。' metadata={'Header 1': 'あらすじ４'}
page_content='魔王を倒し王都へがい旋した勇者ヒンメル一行。' metadata={'Header 1': '第1話 葬送のフリーレン「冒険の終わり」'}
page_content='魔王を倒し王都へがい旋した勇者ヒンメル一行。' metadata={'Header 1': '第1話 葬送のフリーレン「冒険の終わり」'}
page_content='交易都市ヴァルムへとやって来たフリーレンとフェルン。' metadata={'Header 1': '第3話 葬送のフリーレン「人を殺す魔法」'}
page_content='交易都市ヴァルムへとやって来たフリーレンとフェルン。' metadata={'Header 1': '第3話 葬送のフリーレン「人を殺す魔法」'}
page_content='\nそしてアイゼンの弟子シュタルクや僧侶ザインを仲間に加え、フリーレン達の旅は続く' metadata={'Header 1': 'あらすじ４'}
page_content='\nそしてアイゼンの弟子シュタルクや僧侶ザインを仲間に加え、フリーレン達の旅は続く' metadata={'Header 1': 'あらすじ４'}
page_content='フリーレンとフェルンはアイゼンを訪ねる。' metadata={'Header 1': '第4話 葬送のフリーレン「魂の眠る地」'}
page_content='フリーレンとフェルンはアイゼンを訪ねる。' metadata={'Header 1': '第4話 葬送のフリーレン「魂の眠る地」'}
------

db2 oshizo/sbert-jsnli-luke-japanese-base-lite
10
page_content='当ての無かった旅は、ヒンメルとの再会を目指す北への旅となる。' metadata={'Hea

In [ ]:
# 使わない
with open("./test/text_chunks.txt", "w", encoding="utf-8") as file:
    for text in documents:
        file.write(text.page_content)
        file.write('\n--------------------------------------\n')


In [ ]:
# 使わない（もともと最初に作ったやつ）
from langchain.embeddings import HuggingFaceEmbeddings

db = Chroma.from_documents(
    documents=documents,
    #  InvalidDimensionException: Dimensionality of (1536) does not match index dimensionality (384)
    # embeddingを変更したらいったんDBのフォルダごと削除
    # embedding=OpenAIEmbeddings(),
    # embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite" , encode_kwargs={"normalize_embeddings":True }),
    embedding = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-small', encode_kwargs={"normalize_embeddings":True }),
    persist_directory='testdb'
)
if db:
    db.persist()
    # db = None
else:
    print("Chroma DB has not been initialized.")

# c:\Users\onowa\miniforge3\envs\env_chroma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

db2 = Chroma.from_documents(
    documents=documents,
    #  InvalidDimensionException: Dimensionality of (1536) does not match index dimensionality (384)
    # embeddingを変更したらいったんDBのフォルダごと削除
    # embedding=OpenAIEmbeddings(),
    embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite" , encode_kwargs={"normalize_embeddings":True }),
    # embedding = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-small', encode_kwargs={"normalize_embeddings":True }),
    persist_directory='testdb2'
)
if db2:
    db2.persist()
    # db = None
else:
    print("Chroma DB has not been initialized.")

# c:\Users\onowa\miniforge3\envs\env_chroma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db3 = Chroma.from_documents(
    documents=documents,
    #  InvalidDimensionException: Dimensionality of (1536) does not match index dimensionality (384)
    # embeddingを変更したらいったんDBのフォルダごと削除
    # embedding=OpenAIEmbeddings(),
    # embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite" , encode_kwargs={"normalize_embeddings":True }),
    # embedding = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-small', encode_kwargs={"normalize_embeddings":True }),
    embedding = embedding_function,
    persist_directory='testdb3'
)

if db3:
    db3.persist()
    # db = None
else:
    print("Chroma DB has not been initialized.")

# c:\Users\onowa\miniforge3\envs\env_chroma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db4 = Chroma.from_documents(
    documents=documents,
    # embeddingを変更したらいったんDBのフォルダごと削除
    # embedding=OpenAIEmbeddings(),
    # embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite" , encode_kwargs={"normalize_embeddings":True }),
    embedding = HuggingFaceEmbeddings(model_name = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2', encode_kwargs={"normalize_embeddings":True }),
    # embedding = embedding_function,
    persist_directory='testdb4'
)

if db4:
    db4.persist()
    # db = None
else:
    print("Chroma DB has not been initialized.")

# c:\Users\onowa\miniforge3\envs\env_chroma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#  from .autonotebook import tqdm as notebook_tqdm


In [50]:
# load from disk
del db2
list_db = [
    {'db':'db1', 'embedding_function':'sentence-transformers/all-MiniLM-L6-v2'},
    {'db':'db2', 'embedding_function':'oshizo/sbert-jsnli-luke-japanese-base-lite'},
    {'db':'db3', 'embedding_function':'intfloat/multilingual-e5-small'},
    {'db':'db4', 'embedding_function':'sonoisa/sentence-bert-base-ja-mean-tokens-v2'}
]

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name='oshizo/sbert-jsnli-luke-japanese-base-lite')
db2 = Chroma(persist_directory="./database/db2", embedding_function=embedding_function)
query = "アイゼンの弟子シュタルクや僧侶ザイン"
docs = db2.similarity_search(query)
print(docs[0].page_content)


そしてアイゼンの弟子シュタルクや僧侶ザインを仲間に加え、フリーレン達の旅は続く


In [ ]:
# データの検索
# similarity_search, similarity_search_with_scoreなど、いくつかのメソッドがある。

docs = db.similarity_search("フリーレン", top_k=2)  # top_kはデフォルト4
print(docs[0].page_content)  
print(docs[1].page_content)  

docs = db.similarity_search_with_score("フリーレン")
print(len(docs))
print(f"content: {docs[0][0].page_content}", f"score: {docs2[0][1]}")  # content: メロスは激怒した。 score: 0.5952237248420715
print(f"content: {docs[3][0].page_content}", f"score: {docs2[1][1]}")  
for mykey in docs:
    print(mykey)

docs2 = db2.similarity_search_with_score("フリーレン")
print(len(docs2))
print(f"content: {docs2[0][0].page_content}", f"score: {docs2[0][1]}")  # content: メロスは激怒した。 score: 0.5952237248420715
print(f"content: {docs2[3][0].page_content}", f"score: {docs2[1][1]}")  
for mykey in docs2:
    print(mykey)

docs3 = db3.similarity_search_with_score("フリーレン")
print(len(docs3))
print(f"content: {docs3[0][0].page_content}", f"score: {docs2[0][1]}")  # content: メロスは激怒した。 score: 0.5952237248420715
print(f"content: {docs3[3][0].page_content}", f"score: {docs2[1][1]}")  
for mykey in docs3:
    print(mykey)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
texts = text_splitter.split_documents(data)  # (documents)
embeddings = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite" ),
vectordb = Chroma.from_documents(documents, embeddings) #(texts, embeddings)